<a href="https://colab.research.google.com/github/hugopoggi/enem/blob/main/impacto_pandemia_enem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install pyspark

In [93]:
from pyspark.sql import SparkSession as ss
from pyspark.sql import functions as f
from pyspark.sql.types import DoubleType

In [94]:
spark = ss.builder.appName("ENEM").getOrCreate()

**INICIANDO TRATAMENTO DE DADOS ENEM 2019
**

In [95]:
# Importando a base de dados do enem de 2019
url = "/content/drive/MyDrive/BasesSpark/enem/enem_2019/DADOS/MICRODADOS*"
enem_2019 =spark.read.csv(
    url,
    header=True,
    inferSchema=True,
    encoding= "UTF-8",
    sep=";"
)

In [96]:
enem_2019.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_AN

In [97]:
enem_2019.count()

5095171

In [98]:
enem_2019.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [99]:
# Filtrando alunos que estiveram presentes em todos os dias de prova, que nao concluiram ou irão concluir o ensino medio
enem_2019.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [100]:
# Aplicando alteração ao Data Frame
enem_2019 = enem_2019.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
)

In [101]:
enem_2019.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [102]:
# Removendo notas nulas:
enem_2019.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"]).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [103]:
# Fazendo backup
bkp = enem_2019

In [104]:
# Aplicando alteração:
enem_2019 = enem_2019.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"])

In [105]:
# Validando a alteração: verificando se existem valores nulos nas colunas especificadas
enem_2019.filter((enem_2019["NU_NOTA_CN"].isNull()) |
                 (enem_2019["NU_NOTA_CH"].isNull()) |
                 (enem_2019["NU_NOTA_LC"].isNull()) |
                 (enem_2019["NU_NOTA_MT"].isNull()) |
                 (enem_2019["NU_NOTA_REDACAO"].isNull())).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------+---------------+---------------+---------------+---------+--------------+--------------+--------------+--------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|

In [106]:
# Filtrando colunas que não interessam para esta analise:
enem_2019.drop("TP_ANO_CONCLUIU",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025").show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ESCOLA|IN_TREINEIRO|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|  NO_MUNICIPIO_PROVA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|190001592266|  2019|              2|      M|              1|          1|               1|        2|           0|     NULL|                  NULL|              NULL|           S�o Paulo|     543.9|     548.1|     502.5|     480.7|            400|
|19000142154

In [107]:
# Aplicando Data frame
enem_2019 = enem_2019.drop(
               "TP_ANO_CONCLUIU",
               "TP_ESTADO_CIVIL",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025")

In [108]:
enem_2019.show()

+------------+------+---------------+-------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_COR_RACA|TP_NACIONALIDADE|TP_ESCOLA|IN_TREINEIRO|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|  NO_MUNICIPIO_PROVA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+------+---------------+-------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|190001592266|  2019|              2|      M|          1|               1|        2|           0|     NULL|                  NULL|              NULL|           S�o Paulo|     543.9|     548.1|     502.5|     480.7|            400|
|190001421547|  2019|              4|      M|          3|               1|  

In [109]:
# Renomenando colunas:
enem_2019.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
}).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|           2|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|           4|     M|       3|            1|          1|        1|     NULL|         

In [110]:
# Aplicando alterações
enem_2019=enem_2019.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
})

In [111]:
enem_2019.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|           2|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|           4|     M|       3|            1|          1|        1|     NULL|         

In [112]:
# Alterando valores das colunas segundo dicionario de dados
enem_2019.withColumn(    "faixa_etaria",
                         f.when(enem_2019.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2019.faixa_etaria == 2, "17")\
                         .when(enem_2019.faixa_etaria == 3, "18")\
                         .when(enem_2019.faixa_etaria == 4, "19")\
                         .when(enem_2019.faixa_etaria == 5, "20")\
                         .when(enem_2019.faixa_etaria == 6, "21")\
                         .when(enem_2019.faixa_etaria == 7, "22")\
                         .when(enem_2019.faixa_etaria == 8, "23")\
                         .when(enem_2019.faixa_etaria == 9, "24")\
                         .when(enem_2019.faixa_etaria == 10, "25").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|       3|            1|          1|        1|     NULL|         

In [113]:
# Implementando alteração:
enem_2019 = enem_2019.withColumn(    "faixa_etaria",
                         f.when(enem_2019.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2019.faixa_etaria == 2, "17")\
                         .when(enem_2019.faixa_etaria == 3, "18")\
                         .when(enem_2019.faixa_etaria == 4, "19")\
                         .when(enem_2019.faixa_etaria == 5, "20")\
                         .when(enem_2019.faixa_etaria == 6, "21")\
                         .when(enem_2019.faixa_etaria == 7, "22")\
                         .when(enem_2019.faixa_etaria == 8, "23")\
                         .when(enem_2019.faixa_etaria == 9, "24")\
                         .when(enem_2019.faixa_etaria == 10, "25").otherwise("N/I")
                         )

In [114]:
enem_2019.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|       3|            1|          1|        1|     NULL|         

In [115]:
# Alterando valores das colunas segundo dicionario de dados
enem_2019.withColumn("cor_raca",
                         f.when(enem_2019.cor_raca == 1, "BRANCA")\
                         .when(enem_2019.cor_raca == 2, "PRETA")\
                         .when(enem_2019.cor_raca == 3, "PARDA")\
                         .when(enem_2019.cor_raca == 4, "AMARELA")
                         .when(enem_2019.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|            1|          1|        1|     NULL|         

In [116]:
# Implementando alteração
enem_2019 = enem_2019.withColumn("cor_raca",
                         f.when(enem_2019.cor_raca == 1, "BRANCA")\
                         .when(enem_2019.cor_raca == 2, "PRETA")\
                         .when(enem_2019.cor_raca == 3, "PARDA")\
                         .when(enem_2019.cor_raca == 4, "AMARELA")
                         .when(enem_2019.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         )

In [117]:
enem_2019.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|            1|          1|        1|     NULL|         

In [118]:
# Alterando valores de colunas segundo dicionario
enem_2019.withColumn("nacionalidade",
                         f.when(enem_2019.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2019.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2019.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2019.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|       BRASILEIRO(A)|      

In [119]:
# Implementando normalização:
enem_2019 = enem_2019.withColumn("nacionalidade",
                         f.when(enem_2019.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2019.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2019.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2019.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         )

In [120]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|       BRASILEIRO(A)|      

In [121]:
# Alterando valores das colunas segundo dicionario:
enem_2019.withColumn("tipo_escola",
                         f.when(enem_2019.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2019.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2019.tipo_escola == 4, "EXTERIOR")
).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|       BRASILEIRO(A)|      

In [122]:
enem_2019 = enem_2019.withColumn("tipo_escola",
                         f.when(enem_2019.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2019.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2019.tipo_escola == 4, "EXTERIOR")
)

In [123]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|       BRASILEIRO(A)|      

In [124]:
# Removendo valores nulos do tipo escola:
enem_2019.dropna(subset=["tipo_escola"]).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PR

In [125]:
# Aplicando alteração:
enem_2019 = enem_2019.dropna(subset=["tipo_escola"])

In [126]:
# Verificando alteração:
enem_2019.filter((enem_2019["tipo_escola"].isNull())).show()

+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
|inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+



In [127]:
# Alterando dados segundo dicionario
enem_2019.withColumn("treineiro",
                         f.when(enem_2019.treineiro == 1, "SIM").otherwise("NAO")
                         ).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|      NAO|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PR

In [128]:
enem_2019=enem_2019.withColumn("treineiro",
                         f.when(enem_2019.treineiro == 1, "SIM").otherwise("NAO"))

In [129]:
# Atualizando backup
bpk = enem_2019

In [130]:
# Tratando dados "uf_escola" e "municipio_esc" - se "uf_escola" e "municipio_esc"
# for nulo remova a linha
enem_2019.dropna(thresh=1,subset=["uf_escola","dependencia_esc"]).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|              4|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [131]:
enem_2019 = enem_2019.dropna(thresh=2,subset=["uf_escola","dependencia_esc"])

In [132]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|              4|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [133]:
# Alterando dados de colunas segundo dicionario:
enem_2019.withColumn("dependencia_esc",
                         f.when(enem_2019.dependencia_esc == 1, "FEDERAL")\
                         .when(enem_2019.dependencia_esc == 2, "MUNICIPAL")\
                         .when(enem_2019.dependencia_esc == 3, "PRIVADA").otherwise("N/I")
).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [134]:
enem_2019 = enem_2019.withColumn("dependencia_esc",
                         f.when(enem_2019.dependencia_esc == 1, "FEDERAL")\
                         .when(enem_2019.dependencia_esc == 2, "MUNICIPAL")\
                         .when(enem_2019.dependencia_esc == 3, "PRIVADA").otherwise("N/I")
)

In [135]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [136]:
# Alterando dados de colunas segundo dicionario:
enem_2019.withColumn("localizacao_esc",
                         f.when(enem_2019.localizacao_esc == 1, "URBANO")\
                         .when(enem_2019.localizacao_esc == 2, "RURAL")
).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [137]:
enem_2019 = enem_2019.withColumn("localizacao_esc",
                         f.when(enem_2019.localizacao_esc == 1, "URBANO")\
                         .when(enem_2019.localizacao_esc == 2, "RURAL")
)

In [138]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [139]:
bkp= enem_2019

In [140]:
# Removendo dados nulos da coluna "localizacao_esc":
enem_2019 = enem_2019.dropna(subset=['localizacao_esc'])

In [141]:
# Verificando alteração
enem_2019.filter((enem_2019['localizacao_esc'].isNull())).show()

+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
|inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+



In [142]:
bkp = enem_2019

In [143]:
# Adicionando a coluna media final para enem:
enem_2019 = enem_2019.withColumn("nota_final",
                                  (f.col("n_ciencias_nat") +
                                   f.col("n_ciencias_hum") +
                                   f.col("n_linguagens_cod") +
                                   f.col("n_matematica") +
                                   f.col("n_redacao")) / 5)

In [144]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+------------------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|        nota_final|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+------------------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|            722.58|
|190

In [145]:
bkp = enem_2019

In [146]:
# normalizando coluna nota final deixando 2 casa apos o . :
enem_2019 = enem_2019.withColumn("nota_final",
                                 f.format_number("nota_final", 2))

In [147]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|nota_final|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|    722.58|
|190001692704|     2019|          18

CONSULTAS SQL:


In [148]:
enem_2019.count()

940926

In [149]:
# Criando view
enem_2019.createOrReplaceTempView("v_enem_2019")

In [150]:
spark.sql(
"""
SELECT A.uf_escola, ROUND(AVG(A.nota_final), 2) AS media_estado
FROM v_enem_2019 A
GROUP BY uf_escola
ORDER BY media_estado DESC
"""
).toPandas()

,uf_escola,media_estado
0,DF,542.84
1,RJ,542.60
2,RS,541.38
3,MG,540.90
4,SC,539.17
5,SP,538.54
6,ES,537.31
7,PR,533.68
8,GO,528.88
9,RN,526.97


In [151]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|nota_final|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|    722.58|
|190001692704|     2019|          18

In [152]:
spark.sql(
"""
SELECT A.municipio_esc,
COUNT(inscricao) AS total_inscricoes,
ROUND(AVG(nota_final), 2) as media_nota
FROM v_enem_2019 AS A
GROUP BY municipio_esc
ORDER BY total_inscricoes DESC, media_nota DESC
"""
).toPandas()

,municipio_esc,total_inscricoes,media_nota
0,S�o Paulo,41332,542.73
1,Rio de Janeiro,23575,561.74
2,Fortaleza,22659,523.71
3,Bras�lia,18145,542.97
4,Manaus,12909,498.41
...,...,...,...
1687,Porto Walter,19,489.35
1688,Canutama,19,463.26
1689,Fernando de Noronha,10,519.08
1690,Santa Rosa do Purus,5,482.14


In [153]:
spark.sql(
"""
SELECT A.uf_escola,
COUNT(inscricao) AS total_inscricoes,
ROUND(AVG(nota_final), 2) as media_nota
FROM v_enem_2019 AS A
GROUP BY uf_escola
ORDER BY total_inscricoes DESC, media_nota DESC
"""
).toPandas()

,uf_escola,total_inscricoes,media_nota
0,SP,175880,538.54
1,MG,90767,540.90
2,CE,86267,494.21
3,RJ,60938,542.60
4,BA,58818,507.92
5,PE,50617,511.08
6,PR,42171,533.68
7,PA,39039,500.12
8,RS,38669,541.38
9,MA,35959,484.08


In [154]:
spark.sql(
    """
    SELECT A.cor_raca,
           COUNT(A.inscricao) AS total_inscricoes,
           ROUND(AVG(nota_final), 2) as media_final
    FROM v_enem_2019 AS A
    GROUP BY A.cor_raca
    ORDER BY total_inscricoes DESC
    """
).toPandas()

,cor_raca,total_inscricoes,media_final
0,PARDA,426456,505.25
1,BRANCA,362858,549.57
2,PRETA,105626,502.04
3,AMARELA,21131,520.92
4,N/I,19275,512.64
5,INDIGENA,5580,479.79


In [155]:
spark.sql(
    """
SELECT A.inscricao AS numero_inscricao,
       A.municipio_esc AS municipio,
       A.n_ciencias_nat AS ciencias_naturais,
       A.n_ciencias_hum AS ciencias_humanas,
       A.n_linguagens_cod AS linguagens_codigos,
       A.n_matematica AS matematica,
       A.n_redacao AS redacao,
       A.nota_final
FROM v_enem_2019 AS A
WHERE nota_final > 100
ORDER BY nota_final DESC
    """
).toPandas()

,numero_inscricao,municipio,ciencias_naturais,ciencias_humanas,linguagens_codigos,matematica,redacao,nota_final
0,190003517496,Belo Horizonte,772.3,768.3,749.3,984.2,980,850.82
1,190002821921,Pouso Alegre,842.8,733.4,690.9,984.2,980,846.26
2,190001099380,Teresina,843.2,789.9,627.7,984.2,980,845.00
3,190002879782,Goi�nia,813.1,767.4,738.6,914.4,980,842.70
4,190004127950,Goi�nia,829.4,752.1,657.1,984.2,980,840.56
...,...,...,...,...,...,...,...,...
940917,190001158964,Quiterian�polis,363.3,0.0,0.0,376.0,0,147.86
940918,190003014626,Cuiab�,404.3,0.0,322.0,0.0,0,145.26
940919,190001366880,Recife,384.5,0.0,324.0,0.0,0,141.70
940920,190003609385,Juazeiro do Norte,0.0,0.0,0.0,0.0,620,124.00


---





**Importando e iniciando tratamento de dados Data Frame enem_2020**

---




In [156]:
# Importando a base de dados do enem de 2020
url = "/content/drive/MyDrive/BasesSpark/enem/enem_2020/DADOS/MICRODADOS*"
enem_2020 =spark.read.csv(
    url,
    header=True,
    inferSchema=True,
    encoding= "UTF-8",
    sep=";"
)

In [157]:
enem_2020.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|

In [158]:
enem_2020.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [159]:
enem_2020.count()

5783109

In [160]:
# Filtrando alunos que estiveram presentes em todos os dias de prova, que nao concluiram ou irão concluir o ensino medio
enem_2020.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [161]:
enem_2020 =enem_2020.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
)

In [162]:
# Removendo notas nulas:
enem_2020.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"]).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [163]:
# Removendo notas nulas:
enem_2020= enem_2020.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"])

In [164]:
# Validando a alteração: verificando se existem valores nulos nas colunas especificadas
enem_2020.filter((enem_2020["NU_NOTA_CN"].isNull()) |
                 (enem_2020["NU_NOTA_CH"].isNull()) |
                 (enem_2020["NU_NOTA_LC"].isNull()) |
                 (enem_2020["NU_NOTA_MT"].isNull()) |
                 (enem_2020["NU_NOTA_REDACAO"].isNull())).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------+---------------+---------------+---------------+---------+--------------+--------------+--------------+--------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|

In [165]:
# Filtrando colunas que não interessam para esta analise:
enem_2020.drop("TP_ANO_CONCLUIU",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025").show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+-------------------+----------+----------+----------+----------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ESCOLA|IN_TREINEIRO|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC| NO_MUNICIPIO_PROVA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+-------------------+----------+----------+----------+----------+---------------+
|200001908998|  2020|              2|      M|              1|          3|               1|        2|           0|       SP|                     3|                 1|Santana de Parna�ba|     620.8|     675.0|     624.2|     759.4|            760|
|200003209996|  

In [179]:
enem_2020 = enem_2020.drop(
               "TP_ANO_CONCLUIU",
               "TP_ESTADO_CIVIL",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025")

In [180]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|            1|          2|        0|       AM|             

In [181]:
# Renomenando colunas:
enem_2020.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
}).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|            1|          2|        0|       AM|             

In [182]:
# Aplicando alterações
enem_2020=enem_2020.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
})

In [183]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|            1|          2|        0|       AM|             

In [184]:
# Alterando valores das colunas segundo dicionario de dados
enem_2020.withColumn(    "faixa_etaria",
                         f.when(enem_2020.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2020.faixa_etaria == 2, "17")\
                         .when(enem_2020.faixa_etaria == 3, "18")\
                         .when(enem_2020.faixa_etaria == 4, "19")\
                         .when(enem_2020.faixa_etaria == 5, "20")\
                         .when(enem_2020.faixa_etaria == 6, "21")\
                         .when(enem_2020.faixa_etaria == 7, "22")\
                         .when(enem_2020.faixa_etaria == 8, "23")\
                         .when(enem_2020.faixa_etaria == 9, "24")\
                         .when(enem_2020.faixa_etaria == 10, "25").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|   PARDA|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|   PARDA|            1|          2|        0|       AM|             

In [185]:
# Aplicando valores
enem_2020 =  enem_2020.withColumn(    "faixa_etaria",
                         f.when(enem_2020.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2020.faixa_etaria == 2, "17")\
                         .when(enem_2020.faixa_etaria == 3, "18")\
                         .when(enem_2020.faixa_etaria == 4, "19")\
                         .when(enem_2020.faixa_etaria == 5, "20")\
                         .when(enem_2020.faixa_etaria == 6, "21")\
                         .when(enem_2020.faixa_etaria == 7, "22")\
                         .when(enem_2020.faixa_etaria == 8, "23")\
                         .when(enem_2020.faixa_etaria == 9, "24")\
                         .when(enem_2020.faixa_etaria == 10, "25").otherwise("N/I")
                         )

In [186]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|   PARDA|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|   PARDA|            1|          2|        0|       AM|             

In [187]:
# Alterando valores das colunas segundo dicionario de dados
enem_2020.withColumn("cor_raca",
                         f.when(enem_2020.cor_raca == 1, "BRANCA")\
                         .when(enem_2020.cor_raca == 2, "PRETA")\
                         .when(enem_2020.cor_raca == 3, "PARDA")\
                         .when(enem_2020.cor_raca == 4, "AMARELA")
                         .when(enem_2020.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|     N/I|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|     N/I|            1|          2|        0|       AM|             

In [188]:
# Alterando valores das colunas segundo dicionario de dados
enem_2020 = enem_2020.withColumn("cor_raca",
                         f.when(enem_2020.cor_raca == 1, "BRANCA")\
                         .when(enem_2020.cor_raca == 2, "PRETA")\
                         .when(enem_2020.cor_raca == 3, "PARDA")\
                         .when(enem_2020.cor_raca == 4, "AMARELA")
                         .when(enem_2020.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         )

In [189]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|     N/I|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|     N/I|            1|          2|        0|       AM|             

In [190]:
# Alterando valores de colunas segundo dicionario
enem_2020.withColumn("nacionalidade",
                         f.when(enem_2020.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2020.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2020.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2020.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|     N/I|BRASILEIRO(A)|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|     N/I|BRASILEIRO(A)|          2|        0|       AM|             

In [191]:
# Aplicando alteração:
enem_2020=enem_2020.withColumn("nacionalidade",
                         f.when(enem_2020.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2020.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2020.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2020.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         )

In [192]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|     N/I|BRASILEIRO(A)|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|     N/I|BRASILEIRO(A)|          2|        0|       AM|             

In [193]:
# Alterando valores das colunas segundo dicionario:
enem_2020.withColumn("tipo_escola",
                         f.when(enem_2020.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2020.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2020.tipo_escola == 4, "EXTERIOR")
).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|     N/I|BRASILEIRO(A)|    PUBLICA|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|     N/I|BRASILEIRO(A)|    PUBLICA|        0|       AM|             

In [194]:
# Aplicando alterações:
enem_2020 = enem_2020.withColumn("tipo_escola",
                         f.when(enem_2020.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2020.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2020.tipo_escola == 4, "EXTERIOR")
)

In [195]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|     N/I|BRASILEIRO(A)|    PUBLICA|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|     N/I|BRASILEIRO(A)|    PUBLICA|        0|       AM|             

In [197]:
# Removendo valores nulos do tipo escola:
enem_2020.dropna(subset=["tipo_escola"]).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|     N/I|BRASILEIRO(A)|    PUBLICA|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|     N/I|BRASILEIRO(A)|    PUBLICA|        0|       AM|             

In [198]:
# Aplicando alteração
enem_2020 = enem_2020.dropna(subset=["tipo_escola"])

In [199]:
# Verificando alteração:
enem_2020.filter((enem_2020["tipo_escola"].isNull())).show()

+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
|inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+



In [200]:
# Alterando dados segundo dicionario
enem_2020.withColumn("treineiro",
                         f.when(enem_2020.treineiro == 1, "SIM").otherwise("NAO")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|     N/I|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|     N/I|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|             

In [201]:
enem_2020 = enem_2020.withColumn("treineiro",
                         f.when(enem_2020.treineiro == 1, "SIM").otherwise("NAO")
                         )

In [202]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|         N/I|     M|     N/I|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|         N/I|     F|     N/I|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|             